In [1]:
from environment import Environment
from magiccube import Cube
from agents import A2C, DQN, PPO
import yaml
import numpy as np
import itertools
import torch
import torch.nn as nn

In [2]:
args = yaml.safe_load(open("config.yaml", "r"))

env = Environment(
    method="LBL",
    size=3,
    args=args["environment"]
)

agent = DQN(env, args["DQN"])

In [3]:
cube = Cube()
print(cube)

          W  W  W                   
          W  W  W                   
          W  W  W                   
 O  O  O  G  G  G  R  R  R  B  B  B 
 O  O  O  G  G  G  R  R  R  B  B  B 
 O  O  O  G  G  G  R  R  R  B  B  B 
          Y  Y  Y                   
          Y  Y  Y                   
          Y  Y  Y                   



In [4]:
env.algorithm.status()

array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1]])

In [5]:
#env.scramble()
env.step("B'")
print(env.state2)
#print(env.state2.shape)
#print(env.algorithm.status())
print(env.cube)
print(agent.policy_net(agent.state_to_tensor(env.state2)))

[[[ 3  0  4  0 17]
  [ 1  3 -1  1  1]
  [ 1  3  5  2  2]
  [ 3 -1  4  3  9]
  [ 1 -1 -1  4  4]
  [ 1 -1  5  5  5]
  [ 3  1  4  6  0]
  [ 1  2 -1  7  7]
  [ 1  2  5  8  8]
  [-1  0  4  9 20]
  [-1  3 -1 10 10]
  [-1  3  5 11 11]
  [-1 -1  4 12 12]
  [-1 -1  5 13 13]
  [-1  1  4 14  3]
  [-1  2 -1 15 15]
  [-1  2  5 16 16]
  [ 2  0  4 17 23]
  [ 0  3 -1 18 18]
  [ 0  3  5 19 19]
  [ 2 -1  4 20 14]
  [ 0 -1 -1 21 21]
  [ 0 -1  5 22 22]
  [ 2  1  4 23  6]
  [ 0  2 -1 24 24]
  [ 0  2  5 25 25]]]
          O  O  O                   
          W  W  W                   
          W  W  W                   
 Y  O  O  G  G  G  R  R  W  B  B  B 
 Y  O  O  G  G  G  R  R  W  B  B  B 
 Y  O  O  G  G  G  R  R  W  B  B  B 
          Y  Y  Y                   
          Y  Y  Y                   
          R  R  R                   

tensor([[-0.0099, -0.0903,  0.0343, -0.0841, -0.0074,  0.1467,  0.0945, -0.0848,
          0.1046,  0.0575, -0.0936, -0.0138]], grad_fn=<AddmmBackward0>)


In [6]:
hidden_size = 128

embedding = nn.Linear(5, hidden_size)

attention = nn.MultiheadAttention(
    embed_dim=hidden_size,
    num_heads=4,
    batch_first=True
)

layer_norm = nn.LayerNorm(hidden_size)

ffn = nn.Sequential(
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(inplace=True)
)

output_layer = nn.Sequential(
    nn.Linear(hidden_size * 26, hidden_size * 12),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size * 12, hidden_size * 6),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size * 6, hidden_size * 3),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size * 3, hidden_size),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size, hidden_size // 2),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size // 2, 12)
)


In [7]:
state_tensor = torch.from_numpy(env.state2).float()
embedded = embedding(state_tensor)
attn_output, _ = attention(embedded, embedded, embedded)
attn_output = layer_norm(embedded + attn_output)
ffn_output = ffn(attn_output)
output = layer_norm(attn_output + ffn_output)

flattened = output.view(1, -1)
output = output_layer(flattened)

output

tensor([[ 0.0683, -0.0544, -0.0130, -0.0375, -0.0926,  0.0104, -0.0742,  0.0300,
         -0.1461,  0.0402, -0.0654, -0.0434]], grad_fn=<AddmmBackward0>)

In [8]:
# Definiamo un layer lineare: input con 3 feature, output con 2 feature
fc1 = nn.Linear(in_features=3, out_features=256)
fc2 = nn.Linear(in_features=256, out_features=2)

# Esempio di input: batch di 4 esempi, ciascuno con 3 feature
input_matrix = torch.tensor([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0],
    [10.0, 11.0, 12.0]
])  # shape: (4, 3)

# Passaggio della matrice al layer
output = fc1(input_matrix)
print(output)
output = fc2(output)

print("Output:", output)

tensor([[  0.5382,  -0.2334,  -0.8162,  ...,   1.5008,  -1.6134,  -0.5876],
        [  1.7001,  -1.0798,  -2.6460,  ...,   2.1654,  -4.4640,  -3.0868],
        [  2.8619,  -1.9261,  -4.4758,  ...,   2.8301,  -7.3147,  -5.5861],
        [  4.0238,  -2.7724,  -6.3056,  ...,   3.4947, -10.1653,  -8.0853]],
       grad_fn=<AddmmBackward0>)
Output: tensor([[-0.7044,  0.8536],
        [-1.3133,  2.2796],
        [-1.9222,  3.7057],
        [-2.5311,  5.1317]], grad_fn=<AddmmBackward0>)
